In [8]:
using Oscar
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [9]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [10]:
function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [11]:
function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end
    

convertIncidenceMatrix (generic function with 1 method)

In [12]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 

coneListFormat (generic function with 1 method)

In [29]:
function toric_blowup(s, X, v)
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    
    println("coneList: ", coneList)
    
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    #star = rowMinors(coneList, starIndex)
    star = [coneList[i] for i in starIndex]
    
    println("star: ", star)
    rayMatrix = X.RAYS
    #d = size(rayMatrix, 1) #length of array describing each ray
    
    lattice = X.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        println("c: ", c)
        rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
        rank_c_subcones = [faces[i+1] for i in rank_c_subcone_indices]
        append!(clStar, [findall((cone -> all((i -> i in t), cone)), rank_c_subcones)])
        
#         for i in t
#             subcone = convert.(Int64, filter(elt -> elt != i, t))
#             #println(subcone)
#             if rank(Array(rowMinors(rayMatrix, subcone))) == c
#                 append!(clStar, [subcone])
#             end
#         end
    end
    clStar = unique(clStar)
      
    println("clStar: ", clStar)
    
    n = size(rayMatrix, 1) + 1
    #coneList = filter(t -> !(t in star), coneList)
    #coneList  = rowMinors(coneList, filter(i -> !(i in starIndex), range(1, stop=size(coneList, 1))))
    coneList = filter(x -> !(x in star), coneList)
    
    println("coneList: ", coneList)
    
    if length(s) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                 #append!(newCones, [sort(hcat(t..., s...), dims=2)])
                push!(newCones, sort(push!(t, s[1])))
            end
        end
        println(newCones)
         #newCones = convert.(Array{Int64,2}, newCones)
         #println(newCones)
         #println(convert.(Array{Int64,1}, coneList))
        println(append!(coneList, newCones))
        return fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            push!(newCones, push!(t, n))
        end
    end
    
    final_rays = [(X.RAYS); v]
    final_cones = append!(coneList, newCones)
    
    print("rays: ")
    println(final_rays)
    print("cones: ")
    println(final_cones)
    
    return fulton.NormalToricVariety(INPUT_RAYS = final_rays, INPUT_CONES = final_cones)
end

toric_blowup (generic function with 2 methods)

In [23]:
function toric_blowup(s,X)
    rays = rowMinors(Array(X.RAYS), s)
end

toric_blowup (generic function with 2 methods)

In [20]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0 ; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [30]:
toric_blowup([2,3], tv, [1 -1])

coneList: [[1, 2], [2, 3], [3, 4], [1, 4]]
star: [[2, 3]]
c: 1
clStar: Any[[3, 4]]
coneList: [[1, 2], [3, 4], [1, 4]]
rays: pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2
1 -1

cones: [[1, 2], [3, 4], [1, 4], [3, 4, 5]]


type: NormalToricVariety

INPUT_CONES
	{1 2}
	{3 4}
	{1 4}
	{3 4 5}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-1 1/2
	1 -1

In [113]:
coneList = convertIncidenceMatrix(tv.MAXIMAL_CONES)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
coneList = convert.(Array{Int64, 1}, coneList)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
rayMatrix = tv.RAYS

1×0 LinearAlgebra.Transpose{Any,Array{Any,1}}
1×0 LinearAlgebra.Transpose{Any,Array{Any,1}}


pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [14]:
append!([0], [1 2 3])

4-element Array{Int64,1}:
 0
 1
 2
 3

In [16]:
hi = []
append!(hi, [1,2,3])
print(hi)

Any[1, 2, 3]